In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shared_functions_wlp_wls as shared
import scipy.integrate
import pyccl as ccl

/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/halo_table_cache.py:12: UserWarning: Some of the functionality of the HaloTableCache classrequires h5py to be installed.
  warn("Some of the functionality of the HaloTableCache class"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/user_supplied_ptcl_catalog.py:13: UserWarning: Most of the functionality of the sim_manager sub-package requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Most of the functionality of the sim_manager "
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/download_manager.py:39: UserWarning: Some of the functionality of the DownloadManager requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Some of the functionality of the DownloadManager requires h5py to be installed,\n"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/cached_halo_catalog.py:15: UserWarning: Most of the functionality of the

In [2]:
SURVEY = 'SDSS'
if (SURVEY=='SDSS'):
    import params as pa
elif (SURVEY=='LSST_DESI'):
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet; exiting."
    exit()

Goal: produce the projected correlation function of srcs alone for SDSS shapes

In [3]:
#Initialize the cosmology
OmM = pa.OmC_s + pa.OmB_s; OmB=pa.OmB_s; h0 = pa.HH0_s / 100.; sigma80=pa.sigma8_s; n_s0 = pa.n_s_s;

p = ccl.Parameters(Omega_c = OmM-OmB, Omega_b = OmB, h = h0, sigma8=sigma80, n_s=n_s0)
cosmo = ccl.Cosmology(p)

rho_crit = 3. * 10**10 * pa.mperMpc / (8. * np.pi * pa.Gnewt * pa.Msun)  # Msol h^2 / Mpc^3, for use with M in Msol / h (comoving distances)
rho_m = (OmM) * rho_crit # units of Msol h^2 / Mpc^3 (comoving distances)

In [4]:
# define stellar mass bin limits and z's of bins:
Mslow = 6.8 * 10**9 # This what we get for SDSS srcs
z = 0.28 # Set to the effective redshift for now.

# Define a halo mass range
lgMh = np.linspace(10., 15., 30)
Mh = 10**(lgMh)

In [5]:
HMF= ccl.massfunction.massfunc(cosmo, Mh / h0, 1./ (1. + z), odelta=200.) / h0**3
bh= ccl.massfunction.halo_bias(cosmo, Mh / h0, 1./(1.+z), odelta=200.)

Get $<N_{\rm sat}>$ and $<N_{\rm cen}>$ with our $M_*$ threshold.

In [7]:
Nsat = shared.get_Nsat_Zu(Mh, Mslow, 'tot', 'SDSS')
Ncen = shared.get_Ncen_Zu(Mh, Mslow, 'SDSS')

Now get the 1-halo term of the power spectrum.

In [9]:
nbar= scipy.integrate.simps(HMF* (Ncen + Nsat), np.log10(Mh / h0))

# Get the pair terms for each bin
Ncensat = Ncen * Nsat
# For the sat-sat case have to make sure we don't accidentally set negative
Nsatsat= np.zeros(len(Mh))
for mi in range(0,len(Mh)):
    if (Nsat[mi]>1.0):
        Nsatsat[mi] = Nsat[mi] * (Nsat[mi] -1.)
    else:
        Nsatsat[mi] = 0.

In [10]:
# Get the Fourier transformed NFW
logkmin = -6; kpts =40000; logkmax = 5; 
kvec_FT = np.logspace(logkmin, logkmax, kpts)
# Actually we will use a downsampled version of this:
k = np.logspace(np.log10(kvec_FT[0]), np.log10(kvec_FT[-1]), 40)
y = shared.gety_ls(Mh, k, 'SDSS')

In [12]:
# Get the 1-halo power spectrum in each bin
Pkgg_1h= np.zeros(len(k))
for ki in range(0,len(k)):
    Pkgg_1h[ki] = scipy.integrate.simps(HMF*(Ncensat*y[ki, :] + Nsatsat*y[ki, :]**2), np.log10(Mh /h0)) / nbar**2 
        

Now get the 2halo power spectrum in each bin

In [14]:
# Get the linear power spectrum from CCL in each bin
Pklin = ccl.power.linear_matter_power(cosmo, k * h0, 1. / (1. + z)) * h0**3

# Get the 2-halo power spectrum in each bin
Pkgg_2h = np.zeros(len(k))
for ki in range(0,len(k)):
    Pkgg_2h[ki] = (scipy.integrate.simps(HMF * bh * y[ki, :] * (Ncen+ Nsat), np.log10(Mh /h0)) / nbar)**2 * Pklin[ki]
        

Combine 1 and 2 halo terms and output to file for each bin so we can Fourier transform.

In [15]:
Pgg_tot=np.zeros(len(k))
for ki in range(0,len(k)):
    if k[ki]>10**(-2):
        Pgg_tot[ki] = Pkgg_2h[ki] + Pkgg_1h[ki]
    else:
        Pgg_tot[ki] = Pkgg_2h[ki]
    
# This needs to be interms of the more well-sampled k vector
interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(Pgg_tot))
full_Pgg = np.exp(interp_in_k(np.log(kvec_FT)))
    
save_P1h2h = np.column_stack((kvec_FT, full_Pgg))
np.savetxt('./txtfiles/halofit_Pk/Pgg_srcsrc.txt', save_P1h2h)

Import $\xi_{gg}(r)$ as Fourier transformed using FFTlog from our power spectrum.

In [16]:
r, xigg= np.loadtxt('./txtfiles/halofit_xi/xigg_srcsrc.txt', unpack=True)
for ri in range(0,len(r)):
    if xigg[ri]<10**(-12):
        xigg[ri]=10**(-12)

Project to get $w_{gg}$ .

In [17]:
interp_xi = scipy.interpolate.interp1d(np.log(r), np.log(xigg))
    
rp = np.logspace(-2, 2, 100)
Pi = np.logspace(-5, np.log10(80.), 1000) # use the same projection length as Reid & Spergel 2008

# For each bin get xi in terms of rp and Pi
logxi_2d = np.zeros((len(rp), len(Pi)))
for ri in range(0,len(rp)):
    for pi in range(0,len(Pi)):
        r_2d = np.sqrt( rp[ri]**2 + Pi[pi]**2 )
        logxi_2d[ri, pi] = interp_xi(np.log(r_2d))

# Now project at each rp and in each bin            
wgg= np.zeros(len(rp))
for ri in range(0,len(rp)):
    wgg[ri] = 2.*scipy.integrate.simps(np.exp(logxi_2d[ri,:]), Pi)

Now output what we have calculated

In [18]:
plt.figure()
plt.loglog(rp, wgg, 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}$')
plt.savefig('./plots/w_srcsrc.pdf')
plt.close()

Okay so that's all basically correct, up to small factors, which I think are probably explainable by me guessing at redshifts for each bin. That's also not too surprising, given these quantities are largely related to large scales and large scales are not the problem.